In [13]:
%%time

import numpy as np
import torch
import gc
import copy
from torch import nn
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import random
from tqdm.notebook import tqdm
from NN_models import NN_2_256, NN_8_256, NN_8_64
from dataset import make_reactions_dict
from reaction_energy_calculation import calculate_reaction_energy


def set_random_seed(seed):
    # seed evetyting
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    
set_random_seed(41)

CPU times: user 3.63 ms, sys: 2.87 ms, total: 6.5 ms
Wall time: 10 ms


In [2]:
%%time

# make a single dictionary from the whole dataset
data = make_reactions_dict(path='data')

CPU times: user 9.46 s, sys: 4.54 s, total: 14 s
Wall time: 14.5 s


In [3]:
data[73] # take the easiest reaction H2 = 2H

{'Database': 'MGAE109',
 'Components': array(['H_mgae109', 'H2_mgae109'], dtype='<U20'),
 'Coefficients': tensor([ 2., -1.]),
 'Energy': tensor(109.4900),
 'Grid': tensor([[ -1.1678, -16.1181, -16.0818,  ..., -16.1181, -16.1033, -16.1181],
         [ -1.1678, -16.1181, -14.7017,  ..., -16.1181, -15.3051, -16.1181],
         [ -1.1678, -16.1181, -12.3610,  ..., -16.1181, -13.2385, -16.1181],
         ...,
         [ -8.0004,  -8.0004, -14.2658,  ..., -14.2658,  -8.5148,  -8.5148],
         [ -7.7005,  -7.7005, -13.7295,  ..., -13.7295,  -8.2041,  -8.2041],
         [ -8.0004,  -8.0004, -14.2658,  ..., -14.2658,  -8.5148,  -8.5148]]),
 'Weights': tensor([2.5717e-17, 9.9780e-15, 3.2611e-13,  ..., 2.6055e-02, 2.2680e-02,
         2.6055e-02]),
 'Densities': tensor([[0.3111, 0.0000],
         [0.3111, 0.0000],
         [0.3111, 0.0000],
         ...,
         [0.0003, 0.0003],
         [0.0005, 0.0005],
         [0.0003, 0.0003]]),
 'HF_energies': tensor([-0.1908, -0.4750]),
 'backsplit_ind

In [4]:
def rename_keys(data):
    # turns reaction_data dict keys names into numbers
    
    l = len(data)
    keys = data.keys()
    data_new = {}
    for i, key in zip(range(l), keys):
        data_new[i] = data[key]
    return data_new


def train_split(data, test_size, shuffle=False):
    # returns train and test reaction dictionaries
    if shuffle:
        keys = list(data.keys())
        random.shuffle(keys)
        for i in keys:
            data[keys[i]] = data[i]

    train, test = dict(), dict()
    border = round(len(data.keys()) * (1 - test_size))
    for i in range(len(data.keys())):
        if i <= border:
            train[i] = data[i]
        else:
            test[i] = data[i]
    return rename_keys(train), rename_keys(test)

In [5]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data):

        self.data = data
        
    def __getitem__(self, i):
        i = 1
        self.data[i].pop('Database', None)

        return self.data[i], self.data[i]['Energy']
    
    def __len__(self):
        return len(self.data.keys())


test_size = 0.2

data_train, data_test = train_split(copy.deepcopy(data), test_size, shuffle=True)


train_set = Dataset(data=data_train)
train_dataloader = torch.utils.data.DataLoader(train_set, 
                                               batch_size=None,
                                               num_workers=1,
                                               pin_memory=True,
                                               shuffle=True)

test_set = Dataset(data=data_test)
test_dataloader = torch.utils.data.DataLoader(test_set, 
                                              batch_size=None,
                                              num_workers=1,
                                              pin_memory=True,
                                              shuffle=True)

In [6]:
#standard scaler
lst = []
for i in range(len(data)):
    lst.append(data[i]['Grid'])
    
all_grid_data = torch.cat(lst)
print(all_grid_data.shape)

torch.Size([88862460, 7])


In [7]:
%%time
stdscaler = StandardScaler()
stdscaler.fit(np.array(all_grid_data))

CPU times: user 9.66 s, sys: 1.4 s, total: 11.1 s
Wall time: 11.1 s


StandardScaler()

In [ ]:
from NN_models import NN_2_256, NN_8_256, NN_8_64
device = torch.device('cuda:0') if torch.cuda.is_available else torch.device('cpu')

model = NN_8_256(DFT='SVWN').to(device)
# model.load_state_dict(torch.load('predopt/predopt_8_256_log.param', map_location=device))

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, betas=(0.9, 0.999))

In [ ]:
criterion = nn.MSELoss()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# from GPUtil import showUtilization as gpu_usage

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
import os
log_file_path = 'epoch_training.log'
if os.path.isfile(log_file_path):
    os.remove(log_file_path)

In [ ]:
def train(model, criterion, optimizer, train_loader, test_loader, n_epochs=20, accum_iter=4):
    train_loss_mae = []
    train_loss_mse = []
    test_loss_mae = []
    test_loss_mse = []
    accum_iter = accum_iter 

    for epoch in range(n_epochs):
        print(f'Epoch {epoch+1}')
        # train
        
        model.train()
        progress_bar_train = tqdm(train_loader)
        train_mae_losses_per_epoch = []
        train_mse_losses_per_epoch = []
        for batch_idx, X_batch, y_batch in enumetate(progress_bar_train):
            
            X_batch_grid, y_batch = X_batch['Grid'].to(device), y_batch.to(device)
            predictions = model(X_batch_grid)
            reaction_energy = calculate_reaction_energy(X_batch, predictions, device)
            loss = criterion(reaction_energy, y_batch)
            
            # loss_accumulation
            loss = loss / accum_iter
            
            loss.backward()
            
            if ((batch_idx + 1) % accum_iter == 0) or (batch_idx + 1 == len(data_loader)):
                optimizer.step()
                optimizer.zero_grad()

            MSE = loss.item()
            MAE = torch.abs(reaction_energy - y_batch).item()
            train_mse_losses_per_epoch.append(MSE)
            train_mae_losses_per_epoch.append(MAE)
            progress_bar_train.set_postfix(MSE = MSE, MAE = MAE)
            preds = predictions[0].cpu().detach().numpy()
            
            with open('epoch_training.log', 'a') as f:
                f.write(f"{X_batch['Components']} pred {reaction_energy.item():4f} true {y_batch.item():4f} MSE {MSE:4f} MAE {MAE:4f}\n")
                # f.write(f"{preds}\n")
            
            del X_batch, X_batch_grid, y_batch, predictions, reaction_energy, loss, MAE, MSE
            gc.collect()
            torch.cuda.empty_cache()
            # gpu_usage()
        
        train_loss_mse.append(np.mean(train_mse_losses_per_epoch))        
        train_loss_mae.append(np.mean(train_mae_losses_per_epoch))

        print(f'train MSE Loss = {train_loss_mse[epoch]:.8f} MAE Loss = {train_loss_mae[epoch]:.8f}')


        
        #test
        # progress_bar_test = tqdm(test_loader)
        model.eval()
        test_mae_losses_per_epoch = []
        test_mse_losses_per_epoch = []
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                X_batch_grid, y_batch = X_batch['Grid'].to(device), y_batch.to(device)
                predictions = model(X_batch_grid)
                reaction_energy = calculate_reaction_energy(X_batch, predictions, device)
                loss = criterion(reaction_energy, y_batch)
                MSE = loss.item()
                MAE = torch.abs(reaction_energy - y_batch).item()
                progress_bar_train.set_postfix(MAE = MAE)
                test_mse_losses_per_epoch.append(MSE)
                test_mae_losses_per_epoch.append(MAE)
                
                with open('epoch_training.log', 'a') as f:
                    f.write(f"{X_batch['Components']} pred {reaction_energy.item():4f} true {y_batch.item():4f} MSE {MSE:4f} MAE {MAE:4f}\n")
                
                del X_batch, X_batch_grid, y_batch, predictions, reaction_energy, loss, MAE, MSE
                gc.collect()
                torch.cuda.empty_cache()
                
        test_loss_mse.append(np.mean(test_mse_losses_per_epoch))
        test_loss_mae.append(np.mean(test_mae_losses_per_epoch))

        print(f'test MSE Loss = {test_loss_mse[epoch]:.8f} MAE Loss = {test_loss_mae[epoch]:.8f}')

    return train_loss_mae, preds

In [ ]:
from importlib import reload
import SVWN3
reload(SVWN3)

In [ ]:
train_loss_mae, preds = train(model, criterion, optimizer, 
                              train_dataloader,
                              test_dataloader, n_epochs=10)

In [ ]:
train_loss_mae

In [ ]:
def test_lda(train_loader, test_loader, c_arr, criterion, n_epochs=1):
    train_loss_mae = []
    train_loss_mse = []
    test_loss_mae = []
    test_loss_mse = []

    for epoch in range(n_epochs):
        print(f'Epoch {epoch+1}')
        # train
        
        model.train()
        progress_bar_train = tqdm(train_loader)
        train_mae_losses_per_epoch = []
        train_mse_losses_per_epoch = []
        for X_batch, y_batch in progress_bar_train:
            # print(f"{X_batch['Components']}")
            X_batch_grid, y_batch = X_batch['Grid'].to(device), y_batch.to(device)
            # print(torch.tile(c_arr, [X_batch_grid.shape[0],1]))
            predictions = torch.tile(c_arr, [X_batch_grid.shape[0],1]).to(device)
            reaction_energy = calculate_reaction_energy(X_batch, predictions, device)
            loss = criterion(reaction_energy, y_batch)
            MSE = loss.item()
            MAE = torch.abs(reaction_energy - y_batch).item()
            train_mse_losses_per_epoch.append(MSE)
            train_mae_losses_per_epoch.append(MAE)
            progress_bar_train.set_postfix(MSE = MSE, MAE = MAE)
            print(f"{X_batch['Components']} pred {reaction_energy.item():4f} true {y_batch.item():4f} MSE {MSE:4f} MAE {MAE:4f}")

            del X_batch, X_batch_grid, y_batch, predictions, reaction_energy, loss, MAE, MSE
            gc.collect()
            torch.cuda.empty_cache()

        
        train_loss_mse.append(np.mean(train_mse_losses_per_epoch))        
        train_loss_mae.append(np.mean(train_mae_losses_per_epoch))

        print(f'train MSE Loss = {train_loss_mse[epoch]:.8f} MAE Loss = {train_loss_mae[epoch]:.8f}')


        
        #test
        # progress_bar_test = tqdm(test_loader)
        model.eval()
        test_mae_losses_per_epoch = []
        test_mse_losses_per_epoch = []
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                # print(f"{X_batch['Components']}")
                X_batch_grid, y_batch = X_batch['Grid'].to(device), y_batch.to(device)
                predictions = torch.tile(c_arr, (X_batch_grid.shape[0],1)).to(device)
                reaction_energy = calculate_reaction_energy(X_batch, predictions, device)
                loss = criterion(reaction_energy, y_batch)
                MSE = loss.item()
                MAE = torch.abs(reaction_energy - y_batch).item()
                # progress_bar_train.set_postfix(MAE = MAE)
                test_mse_losses_per_epoch.append(MSE)
                test_mae_losses_per_epoch.append(MAE)
                print(f"{X_batch['Components']} pred {reaction_energy.item():4f} true {y_batch.item():4f} MSE {MSE:4f} MAE {MAE:4f}")
                
                del X_batch, X_batch_grid, y_batch, predictions, reaction_energy, loss, MAE, MSE
                gc.collect()
                torch.cuda.empty_cache()
                
        test_loss_mse.append(np.mean(test_mse_losses_per_epoch))
        test_loss_mae.append(np.mean(test_mae_losses_per_epoch))

        print(f'test MSE Loss = {test_loss_mse[epoch]:.8f} MAE Loss = {test_loss_mae[epoch]:.8f}')

    return train_loss_mae

In [ ]:
from importlib import reload
import reaction_energy_calculation
reload(reaction_energy_calculation)


# import SVWN3

In [ ]:
true_constants = torch.Tensor([0.0310907, 0.01554535, 
                3.72744,   7.06042,
                12.9352,   18.0578,
                -0.10498,  -0.32500,
                0.0310907,  0.01554535,  -1/(6*torch.pi**2),
                13.0720,    20.1231,      1.06835,
                42.7198,   101.578,      11.4813,
                -0.409286,  -0.743294,   -0.228344,
                1])

# true_constants

test_lda(train_dataloader, test_dataloader, true_constants, criterion)